#Homework 1A - Task 24

In this homework we aim to create a JSON file, starting from existing datas in order to be used from Generative Language Models.

So strating from existing datasets we will convert them into a format useful for the evaluation of LLM.

SENTIPOLC (SENTIment POLarity Classification) is a task presented at Evalita 2016, the fourth evaluation campaign of Natural Language Processing and Speech tools for Italian

The main goal of SENTIPOLC is sentiment classification at message level on Italian tweets.

The task is divided into three sub-tasks with an increasing level of complexity.

1. **Subjectivity Classification**: Given a message, decide whether the message is subjective or objective.

2. **Polarity Classification**: Given a message, decide whether the message is of positive, negative, neutral or mixed sentiment (i.e. conveying both a positive and negative sentiment).

3. **Irony Detection**: Given a message, decide whether the message is ironic or not


    id: 122449983151669248

    labels: 1	0	1	0	0	1	1

    text: Intanto la partita per Via Nazionale si complica. #Saccomanni dice che "mica tutti sono Mario #Monti" http://t.co/xPtNz4X7 via @linkiesta

The labels correspond respectively to:

* subj Subjectivity: possible values are 0 and 1.
A subjective tweet will have subj = 1;
an objective tweet subj = 0.

* opos Positive overall polarity: possible values are 0 and 1.
A tweet exhibiting positive polarity will have opos = 1;
a tweet without positive polarity will have opos = 0.

* oneg Negative overall polarity: possible values are 0 and 1.
A tweet exhibiting negative polarity will have neg = 1;
a tweet without negative polarity will have neg = 0.

* iro Irony: possible values are 0 and 1.
A tweet with an ironic twist will have iro = 1,
otherwise iro = 0.

#Imports

In [1]:
import requests
import zipfile
import io
import json
import os

url = ["http://www.di.unito.it/~tutreeb/sentipolc-evalita16/training_set_sentipolc16.csv.zip", "http://www.di.unito.it/~tutreeb/sentipolc-evalita16/test_set_sentipolc16_gold2000.csv.zip"]

for u in url:
  response = requests.get(u)

  if response.status_code == 200:

      with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
          zip_ref.extractall("/content")
      print("File "+u+" succesfully downloaded")
  else:
      print("Error")

File http://www.di.unito.it/~tutreeb/sentipolc-evalita16/training_set_sentipolc16.csv.zip succesfully downloaded
File http://www.di.unito.it/~tutreeb/sentipolc-evalita16/test_set_sentipolc16_gold2000.csv.zip succesfully downloaded


# Extract Dataset

In [20]:
test_path ="/content/test_set_sentipolc16_gold2000.csv"
train_path = "/content/training_set_sentipolc16.csv"


def import_data(path, train=True):
  data=[]
  if train:
    with open(path, 'r') as file:
      for line in file:
        data.append([line[1:19], line[21:35], line[35:-1]])
    return data
  else:
    with open(path, 'r') as file:
      for line in file:
        data.append([line[1:19], line[21:35].replace('"', ''), line[49:-1]])
        break
    return data

train_data = import_data(train_path, train=True)
train_data.pop(0)
test_data = import_data(test_path, train=False)

#First Task JSON

In [3]:
def first_task_json(data, t):

    all_records = []

    for i in range(len(data)):

        record = {
            "id": i,
            "text": data[i][2],
            "choices": ['Oggettiva', 'Soggettiva'],
            "label": data[i][1][0],

        }
        all_records.append(record)

    filename = "first_task_" + t + ".json"

    with open(filename, 'w', encoding='utf-8') as json_file:
        json.dump(all_records, json_file, ensure_ascii=False, indent=4)

    return filename

first_task_train = first_task_json(train_data, "train")
first_task_test = first_task_json(test_data, "test")

#Second Task JSON

In [4]:
def second_task_json(data, t):

    all_records = []

    for i in range(len(data)):

        record = {
            "id": i,
            "text": data[i][2],
            "choices": ["Neutrale" , "Negativo", "Positivo", "Misto"],
            "label": int(data[i][1][2]) + (2 * int(data[i][1][4])),
        }
        all_records.append(record)

    filename = "second_task_" + t + ".json"

    with open(filename, 'w', encoding='utf-8') as json_file:
        json.dump(all_records, json_file, ensure_ascii=False, indent=4)
    return filename

second_task_train = second_task_json(train_data, "train")
second_task_test = second_task_json(test_data, "test")

#Third task JSON

In [5]:
def third_task_json(data, t):

    all_records = []

    for i in range(len(data)):

        record = {
            "id": i,
            "text": data[i][2],
            "choices": ['Non Ironico', 'Ironico'],
            "label": data[i][1][0],
        }
        all_records.append(record)

    filename = "third_task_" + t + ".json"

    with open(filename, 'w', encoding='utf-8') as json_file:
        json.dump(all_records, json_file, ensure_ascii=False, indent=4)

    return filename

third_task_train = third_task_json(train_data, "train")
third_task_test = third_task_json(test_data, "test")

#Show input JSON

In [6]:
path="/content"

for file in os.listdir(path):
  if file.endswith(".json"):
    with open(file, 'r') as f:
        data = json.load(f)
        element = data[0]
        print(element)


{'id': 0, 'text': '"Tra 5 minuti presentazione piano scuola del governo #Renzi. #passodopopasso #labuonascuola Stay tuned"', 'choices': ['Oggettiva', 'Soggettiva'], 'label': '0'}
{'id': 0, 'text': '"Tra 5 minuti presentazione piano scuola del governo #Renzi. #passodopopasso #labuonascuola Stay tuned"', 'choices': ['Non Ironico', 'Ironico'], 'label': '0'}
{'id': 0, 'text': '"Intanto la partita per Via Nazionale si complica. #Saccomanni dice che ""mica tutti sono Mario #Monti"" http://t.co/xPtNz4X7 via @linkiesta"', 'choices': ['Oggettiva', 'Soggettiva'], 'label': '1'}
{'id': 0, 'text': '"Tra 5 minuti presentazione piano scuola del governo #Renzi. #passodopopasso #labuonascuola Stay tuned"', 'choices': ['Neutrale', 'Negativo', 'Positivo', 'Misto'], 'label': 0}
{'id': 0, 'text': '"Intanto la partita per Via Nazionale si complica. #Saccomanni dice che ""mica tutti sono Mario #Monti"" http://t.co/xPtNz4X7 via @linkiesta"', 'choices': ['Non Ironico', 'Ironico'], 'label': '1'}
{'id': 0, 'text

#Create Prompt

In [7]:
prompts1 = {
    "prompt_1":"Considerando la frase {text} questa frase è (Oggettiva) o (Soggettiva) ?",
    "prompt_2":"Considerando l'affermazione {text}, trasmette una prospettiva (Oggettiva) o (Soggettiva) ?",
    "prompt_3":"Considerando l'affermazione {text}, riflette sentimenti (Oggettiva) o (Soggettiva) ? ",
}

prompts2 = {
    "prompt_1":"Data la frase '{text}' questa esprime un sentimento (Neutro), (Negativo), (Positivo) o (Misto)?",
    "prompt_2":"Considerando l'affermazione '{text}', questa esprime un sentimento (Neutro), (Negativo), (Positivo) o (Misto)?",
    "prompt_3":"Considerando l'affermazione '{text}', riflette un sentimento (Neutro), (Negativo), (Positivo) o (Misto)?",
}

prompts3 = {
    "prompt_1":"Considerando la frase '{text}' questa è (Non ironico) o (Ironico)?",
    "prompt_2":"Considerando l'affermazione '{text}', è (Non Ironico) o (Ironico)?",
    "prompt_3":"Considerando l'affermazione '{text}', riflette il contesto (Non Ironico) o (Ironico)?",
}
prompts=[prompts1, prompts2, prompts3 ]

for i in range(len(prompts)):
  output_file = "/content/prompts" + str(i+1) + ".jsonl"

  with open(output_file, 'w') as json_file:
      json.dump(prompts[i], json_file, indent=2)

#Test the prompt

In [8]:
import transformers
import torch
import json
import os
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

model_id = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
tokenizer_llama = AutoTokenizer.from_pretrained(model_id)
model_llama = AutoModelForSequenceClassification.from_pretrained(model_id).to(device)

tokenizer_config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [9]:
# Configuration
runtimeFlag = device #Run on GPU (you can't run GPTQ on cpu)
cache_dir = None # by default, don't set a cache directory. This is automatically updated if you connect Google Drive.
scaling_factor = 1.0 # allows for a max sequence length of 16384*6 = 98304! Unfortunately, requires Colab Pro and a V100 or

In [10]:
!pip install -q -U transformers peft accelerate optimum

!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.9 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 707.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [11]:
!pip install -q -U pdfminer.six # could maybe add pre-built wheels to speed this up.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 14.4 MB/s eta 0:00:00


In [12]:
def compute_results(prompt_path, task_path):

  count_array = []
  correct_array = []

  with open(prompt_path, 'r') as file:

      prompts = json.loads(file.read())

  for i in range(1, len(prompts)+1):

    with open(task_path, 'r') as file: #testare per tutti i file

        inputs = json.loads(file.read())

        count_input = 0
        correct = 0

        for input in inputs:

          template = prompts['prompt_'+ str(i)]

          text_to_insert = input['text']

          compiled_string = template.format(text=text_to_insert)

          token = tokenizer_llama(compiled_string, return_tensors="pt").to(device)

          output = model_llama(**token)

          logits = torch.softmax(torch.tensor(output.logits[0].detach().cpu().numpy()), -1).detach().cpu().numpy()
          count_input += 1

          if str(np.argmax(logits)) == str(input['label']):
            correct += 1

          if count_input==100:

            count_array.append(count_input)
            correct_array.append(correct)

            break
  return  count_array,correct_array

In [13]:
def print_results(correct_array, count_array, task):
  print("File:", task)
  print("Accuracy with prompt 1", correct_array[0] / count_array[0] *100)
  print("Accuracy with prompt 2", correct_array[1] / count_array[1] *100)
  print("Accuracy with prompt 3", correct_array[2] / count_array[2] *100)

In [14]:
path = "/content/"
count_array, correct_array = compute_results(path + "prompts1.jsonl", path + first_task_train)
print_results(correct_array, count_array, first_task_train)

File: first_task_train.json
Accuracy with prompt 1 31.0
Accuracy with prompt 2 32.0
Accuracy with prompt 3 47.0


In [15]:
path = "/content/"
count_array, correct_array = compute_results(path + "prompts2.jsonl", path + second_task_train)
print_results(correct_array, count_array, second_task_train)

File: second_task_train.json
Accuracy with prompt 1 30.0
Accuracy with prompt 2 27.0
Accuracy with prompt 3 31.0


In [16]:
path = "/content/"
count_array, correct_array = compute_results(path + "prompts3.jsonl", path + third_task_train)
print_results(correct_array, count_array, third_task_train)

File: third_task_train.json
Accuracy with prompt 1 31.0
Accuracy with prompt 2 34.0
Accuracy with prompt 3 38.0
